# 日本語形態素解析器SudachiPyを使い複合名詞を取得する方法

## 参考リンク
- https://github.com/WorksApplications/SudachiPy

## コード

### パッケージとトークナイザーの設定

In [1]:
# パッケージのインポート

from sudachipy import tokenizer
from sudachipy import dictionary

In [2]:
# sudachipyのtokenizerとmodeのセット

tokenizer_obj = dictionary.Dictionary().create()
mode = tokenizer.Tokenizer.SplitMode.C

In [3]:
# 出力のテスト

[m.surface() for m in tokenizer_obj.tokenize("国家公務員", mode)]

['国家公務員']

### テスト文とその形態素結果

In [4]:
sent1 = '応用例は自然言語処理（機械翻訳・かな漢字変換・構文解析等）、専門家の推論・判断を模倣するエキスパートシステム、画像データを解析して特定のパターンを検出・抽出したりする画像認識等がある'
print(sent1)

応用例は自然言語処理（機械翻訳・かな漢字変換・構文解析等）、専門家の推論・判断を模倣するエキスパートシステム、画像データを解析して特定のパターンを検出・抽出したりする画像認識等がある


In [5]:
for token in tokenizer_obj.tokenize(sent1, mode):
    print(token.surface(), token.part_of_speech())

応用 ['名詞', '普通名詞', 'サ変可能', '*', '*', '*']
例 ['名詞', '普通名詞', '一般', '*', '*', '*']
は ['助詞', '係助詞', '*', '*', '*', '*']
自然言語処理 ['名詞', '普通名詞', '一般', '*', '*', '*']
（ ['補助記号', '括弧開', '*', '*', '*', '*']
機械 ['名詞', '普通名詞', '一般', '*', '*', '*']
翻訳 ['名詞', '普通名詞', 'サ変可能', '*', '*', '*']
・ ['補助記号', '一般', '*', '*', '*', '*']
かな ['名詞', '普通名詞', '一般', '*', '*', '*']
漢字 ['名詞', '普通名詞', '一般', '*', '*', '*']
変換 ['名詞', '普通名詞', 'サ変可能', '*', '*', '*']
・ ['補助記号', '一般', '*', '*', '*', '*']
構文 ['名詞', '普通名詞', '一般', '*', '*', '*']
解析 ['名詞', '普通名詞', 'サ変可能', '*', '*', '*']
等 ['接尾辞', '名詞的', '一般', '*', '*', '*']
） ['補助記号', '括弧閉', '*', '*', '*', '*']
、 ['補助記号', '読点', '*', '*', '*', '*']
専門家 ['名詞', '普通名詞', '一般', '*', '*', '*']
の ['助詞', '格助詞', '*', '*', '*', '*']
推論 ['名詞', '普通名詞', 'サ変可能', '*', '*', '*']
・ ['補助記号', '一般', '*', '*', '*', '*']
判断 ['名詞', '普通名詞', 'サ変可能', '*', '*', '*']
を ['助詞', '格助詞', '*', '*', '*', '*']
模倣 ['名詞', '普通名詞', 'サ変可能', '*', '*', '*']
する ['動詞', '非自立可能', '*', '*', 'サ行変格', '連体形-一般']
エキスパートシステム ['名詞', '

In [6]:
sent2 = '1990年、3D印刷ともっとも広く関連づけられるPlastics extrusion技術が、Stratasys社により"fused deposition modeling (FDM)"（熱溶解積層法）として商品化された'
print(sent2)

1990年、3D印刷ともっとも広く関連づけられるPlastics extrusion技術が、Stratasys社により"fused deposition modeling (FDM)"（熱溶解積層法）として商品化された


In [7]:
for token in tokenizer_obj.tokenize(sent2, mode):
    print(token.surface(), token.part_of_speech())

1990 ['名詞', '数詞', '*', '*', '*', '*']
年 ['名詞', '普通名詞', '助数詞可能', '*', '*', '*']
、 ['補助記号', '読点', '*', '*', '*', '*']
3 ['名詞', '数詞', '*', '*', '*', '*']
D ['名詞', '普通名詞', '一般', '*', '*', '*']
印刷 ['名詞', '普通名詞', 'サ変可能', '*', '*', '*']
と ['助詞', '格助詞', '*', '*', '*', '*']
もっとも ['副詞', '*', '*', '*', '*', '*']
広く ['形容詞', '一般', '*', '*', '形容詞', '連用形-一般']
関連 ['名詞', '普通名詞', 'サ変可能', '*', '*', '*']
づけ ['動詞', '非自立可能', '*', '*', '下一段-カ行', '未然形-一般']
られる ['助動詞', '*', '*', '*', '助動詞-レル', '連体形-一般']
Plastics ['名詞', '普通名詞', '一般', '*', '*', '*']
  ['空白', '*', '*', '*', '*', '*']
extrusion ['名詞', '普通名詞', '一般', '*', '*', '*']
技術 ['名詞', '普通名詞', '一般', '*', '*', '*']
が ['助詞', '格助詞', '*', '*', '*', '*']
、 ['補助記号', '読点', '*', '*', '*', '*']
Stratasys ['名詞', '普通名詞', '一般', '*', '*', '*']
社 ['名詞', '普通名詞', '助数詞可能', '*', '*', '*']
に ['助詞', '格助詞', '*', '*', '*', '*']
より ['動詞', '一般', '*', '*', '五段-ラ行', '連用形-一般']
" ['補助記号', '一般', '*', '*', '*', '*']
fused ['名詞', '普通名詞', '一般', '*', '*', '*']
  ['空白', '*', '*', '*', '*', '*'

### 複合名詞（接頭辞や接尾辞あるなしなどの調整可能）を含む形での分かち書き

In [8]:
# トークナイザーのラッパーとなる自作関数

def custom_tokenize(sentence):
    tokens = []
    nouns = []
    
    is_continuous = False   
    for i, token in enumerate(sentence):
        word = token.surface()

        # 複合名詞として連結する品詞の選択
        tag_split = token.part_of_speech()
        if tag_split[0] == '接頭辞':
            is_use = True
        elif tag_split[0] == '名詞' and tag_split[1] != '接尾語' and tag_split[2] != '副詞可能':
            is_use = True
        elif tag_split[0] == '接尾辞' and tag_split[1] == '名詞的':
            is_use = True
        else:
            is_use = False       

        # gather adjacent nouns and concat them
        if i == len(sentence) - 1:
            if is_use and is_continuous:
                term_current.append(word)
                tokens.append("".join(term_current))
                nouns.append("".join(term_current))
            else:
                tokens.append(word)        
        else:      
            if is_use:            
                if is_continuous == False:
                    is_continuous = True
                    term_current = [word]
                else:
                    term_current.append(word)
            else:
                if is_continuous == True:
                    is_continuous = False
                    tokens.append("".join(term_current))
                    nouns.append("".join(term_current))
                tokens.append(word)
            
    return tokens, nouns

In [9]:
# テスト文への実行

In [10]:
tokens, noun = custom_tokenize(tokenizer_obj.tokenize(sent1, mode))

print(tokens)
print(noun)

['応用例', 'は', '自然言語処理', '（', '機械翻訳', '・', 'かな漢字変換', '・', '構文解析等', '）', '、', '専門家', 'の', '推論', '・', '判断', 'を', '模倣', 'する', 'エキスパートシステム', '、', '画像データ', 'を', '解析', 'し', 'て', '特定', 'の', 'パターン', 'を', '検出', '・', '抽出', 'し', 'たり', 'する', '画像認識等', 'が', 'ある']
['応用例', '自然言語処理', '機械翻訳', 'かな漢字変換', '構文解析等', '専門家', '推論', '判断', '模倣', 'エキスパートシステム', '画像データ', '解析', '特定', 'パターン', '検出', '抽出', '画像認識等']


In [11]:
tokens, noun = custom_tokenize(tokenizer_obj.tokenize(sent2, mode))

print(tokens)
print(noun)

['1990年', '、', '3D印刷', 'と', 'もっとも', '広く', '関連', 'づけ', 'られる', 'Plastics', ' ', 'extrusion技術', 'が', '、', 'Stratasys社', 'に', 'より', '"', 'fused', ' ', 'deposition', ' ', 'modeling', ' ', '(', 'FDM', ')', '"', '（', '熱溶解積層法', '）', 'と', 'し', 'て', '商品化', 'さ', 'れ', 'た']
['1990年', '3D印刷', '関連', 'Plastics', 'extrusion技術', 'Stratasys社', 'fused', 'deposition', 'modeling', 'FDM', '熱溶解積層法', '商品化']


In [12]:
# 接尾辞がいらないときなどは、連結する品詞選択の箇所を調整する

In [13]:
def custom_tokenize(sentence):
    tokens = []
    nouns = []
    
    is_continuous = False   
    for i, token in enumerate(sentence):
        word = token.surface()

        # 複合名詞として連結する品詞の選択
        tag_split = token.part_of_speech()
        if tag_split[0] == '接頭辞':
            is_use = True
        elif tag_split[0] == '名詞' and tag_split[1] != '接尾語' and tag_split[2] != '副詞可能':
            is_use = True
#         elif tag_split[0] == '接尾辞' and tag_split[1] == '名詞的':
#             is_use = True
        else:
            is_use = False       

        # gather adjacent nouns and concat them
        if i == len(sentence) - 1:
            if is_use and is_continuous:
                term_current.append(word)
                tokens.append("".join(term_current))
                nouns.append("".join(term_current))
            else:
                tokens.append(word)        
        else:      
            if is_use:            
                if is_continuous == False:
                    is_continuous = True
                    term_current = [word]
                else:
                    term_current.append(word)
            else:
                if is_continuous == True:
                    is_continuous = False
                    tokens.append("".join(term_current))
                    nouns.append("".join(term_current))
                tokens.append(word)
            
    return tokens, nouns

In [14]:
tokens, noun = custom_tokenize(tokenizer_obj.tokenize(sent1, mode))

print(tokens)
print(noun)

['応用例', 'は', '自然言語処理', '（', '機械翻訳', '・', 'かな漢字変換', '・', '構文解析', '等', '）', '、', '専門家', 'の', '推論', '・', '判断', 'を', '模倣', 'する', 'エキスパートシステム', '、', '画像データ', 'を', '解析', 'し', 'て', '特定', 'の', 'パターン', 'を', '検出', '・', '抽出', 'し', 'たり', 'する', '画像認識', '等', 'が', 'ある']
['応用例', '自然言語処理', '機械翻訳', 'かな漢字変換', '構文解析', '専門家', '推論', '判断', '模倣', 'エキスパートシステム', '画像データ', '解析', '特定', 'パターン', '検出', '抽出', '画像認識']


### 複合名詞とそれを構成する個別名詞の同時出力
ABCという複合名詞があった場合に、A,B,C,AB,ABCを出力する。極めてレアな複合名詞などを扱うときに情報量を増やすときなどの用途用  
例）熱溶解積層法 => '熱', '溶解', '積層', '法', '熱溶解', '熱溶解積層', '熱溶解積層法'

In [15]:
def get_compound_noun(sentence, has_origins=True):
    nouns = []
    
    is_continuous = False   
    for i, token in enumerate(sentence):
        word = token.surface()

        # 複合名詞として連結する品詞の選択
        tag_split = token.part_of_speech()
        if tag_split[0] == '名詞' and tag_split[1] != '接尾語' and tag_split[2] != '副詞可能':
            is_use = True
        else:
            is_use = False       

        # gather adjacent nouns and concat them
        if i == len(sentence) - 1:
            if is_use and is_continuous:
                term_current.append(word)
                if has_origins:
                    compound = term_current + ["".join(term_current[:i+1]) for i in range(1, len(term_current))]
                    nouns.append(compound)
                else:
                    nouns.append("".join(term_current))
                
        else:      
            if is_use:            
                if is_continuous == False:
                    is_continuous = True
                    term_current = [word]
                else:
                    term_current.append(word)
            else:
                if is_continuous == True:
                    is_continuous = False
                    if has_origins:
                        compound = term_current + ["".join(term_current[:i+1]) for i in range(1, len(term_current))]
                        nouns.append(compound)
                    else:
                        nouns.append("".join(term_current))                    
            
    return nouns

In [16]:
# テスト文への実行

In [17]:
compounds = get_compound_noun(tokenizer_obj.tokenize(sent1, mode))

print(compounds)

[['応用', '例', '応用例'], ['自然言語処理'], ['機械', '翻訳', '機械翻訳'], ['かな', '漢字', '変換', 'かな漢字', 'かな漢字変換'], ['構文', '解析', '構文解析'], ['専門家'], ['推論'], ['判断'], ['模倣'], ['エキスパートシステム'], ['画像データ'], ['解析'], ['特定'], ['パターン'], ['検出'], ['抽出'], ['画像', '認識', '画像認識']]


In [18]:
compounds = get_compound_noun(tokenizer_obj.tokenize(sent2, mode))

print(compounds)

[['1990', '年', '1990年'], ['3', 'D', '印刷', '3D', '3D印刷'], ['関連'], ['Plastics'], ['extrusion', '技術', 'extrusion技術'], ['Stratasys', '社', 'Stratasys社'], ['fused'], ['deposition'], ['modeling'], ['FDM'], ['熱', '溶解', '積層', '法', '熱溶解', '熱溶解積層', '熱溶解積層法'], ['商品化']]


In [19]:
# あとは、出力された名詞をさらに、後処理をしてノイズを除去するなどする

In [ ]:
# spacy ginzaで名詞句を撮ってきた場合

In [22]:
import spacy

nlp = spacy.load('ja_ginza_electra')

In [24]:
doc = nlp(sent1)

for span in doc.noun_chunks:
    print(span)

応用例
自然言語処理
特定
パターンを検出・抽出したりする画像認識等


In [25]:
doc = nlp(sent2)

for span in doc.noun_chunks:
    print(span)

1990年
3D印刷
もっとも広く関連づけられるPlastics extrusion技術
Stratasys社
"fused deposition modeling
